## Define input data

In [1]:
from op_coreutils.duckdb_inmem import init_client
from op_coreutils.partitioned import DataReader
from op_datasets.etl.intermediate.construct import construnct_intermediate_model_batches
from op_datasets.etl.intermediate.udfs import create_duckdb_macros



# Define the input data range.
batches: list[DataReader] = construnct_intermediate_model_batches(
    chains=["op"],
    range_spec="@20241030:+1",
)



# Select input for one date
batch = batches[0]

duckdb_client = init_client()
create_duckdb_macros(duckdb_client)

2024-11-10 11:52:19 [info     ] Loaded env var: OP_ANALYTICS_VAULT
2024-11-10 11:52:20 [info     ] Connecting to OPLABS Clickhouse client...
2024-11-10 11:52:20 [info     ] Loaded 17 items into vault.   
2024-11-10 11:52:44 [info     ] Initialized OPLABS Clickhouse client.
2024-11-10 11:52:45 [info     ] 2024-10-30 data input is ready. chain=op date=2024-10-30
2024-11-10 11:52:45 [info     ] Prepared 1 input batches.      chain=op date=2024-10-30


## Register the input relation as a view and run a duckdb query on it


In [3]:
duckdb_client.register(view_name="transactions", python_object=batch.duckdb_relation("transactions"))
duckdb_client.sql("""

SELECT
    receipt_l1_gas_used ,
    receipt_l1_fee / (
        COALESCE(16*micro(receipt_l1_base_fee_scalar), receipt_l1_fee_scalar) * receipt_l1_gas_price + 
        COALESCE(micro(receipt_l1_blob_base_fee_scalar) * receipt_l1_blob_base_fee, 0)) AS estimated_size
FROM transactions
LIMIT 10
""")

┌─────────────────────┬────────────────────┐
│ receipt_l1_gas_used │   estimated_size   │
│        int64        │       double       │
├─────────────────────┼────────────────────┤
│                NULL │               NULL │
│                1600 │  99.99999999967905 │
│                8205 │  512.8503999996193 │
│                1600 │  99.99999999967905 │
│                1600 │  99.99999999967905 │
│                9450 │  590.6448999995525 │
│                1600 │  99.99999999967905 │
│                1600 │  99.99999999967905 │
│                1600 │  99.99999999967905 │
│                7737 │ 483.57289999966065 │
├─────────────────────┴────────────────────┤
│ 10 rows                        2 columns │
└──────────────────────────────────────────┘